Import basic packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

Import packages for handling Bifrost and netCDF files:

In [ ]:
import helita
from helita.sim.bifrost import BifrostData
from helita.sim import cstagger
from netCDF4 import Dataset

Gather output data from Bifrost simulation:

In [ ]:
# Fetch particular snapshot(s) from a given simulation

path = '/mn/stornext/d9/data/rebecrob/betzy_runs/qs012023_bz000_corks/corks_on_1123/qs012023_bz000' # Simulation root
snp = 1292 # Snapshot

d = BifrostData(path,snap=snp)

# Fetch desired variables

x = d.x
y = d.y
z = d.z

bx = d.get_var('bxc')
by = d.get_var('byc') 
bz = d.get_var('bzc') 

# Calculate secondary variables

bmag = np.sqrt(bx**2 + by**2 + bz**2)
bxs, bys, bzs = bx/bmag, by/bmag, bz/bmag # Unit variables for Flow renderer

# Calculate current density
jx = (cstagger.do(cstagger.do(bz, 'ddyup'), 'ydn') - cstagger.do(cstagger.do(by, 'ddzup'), 'zdn'))
jy = (-cstagger.do(cstagger.do(bz, 'ddxup'), 'xdn') + cstagger.do(cstagger.do(bx, 'ddzup'), 'zdn'))
jz = (cstagger.do(cstagger.do(by, 'ddxup'), 'xdn') - cstagger.do(cstagger.do(bx, 'ddyup'), 'ydn'))

jmag = np.sqrt(jx**2 + jy**2 + jz**2)
j_b = jmag / bmag

Write data to netCDF-CF compliant -- this is readable in Vapor

In [ ]:
nc = Dataset('sampledata_'+str(snp)+'.nc', 'w', format='NETCDF4_CLASSIC')
nc.Conventions = 'CF-1.7' 
nc.createDimension('__time', None)
nc.createDimension('x', x.size)
nc.createDimension('y', y.size)
nc.createDimension('z', z.size)


jb_var = nc.createVariable('j_b', datatype=np.float32,dimensions=('x','y','z'))
jb_var[:] = j_b
jb_var.units = 'Bifrost'

bx_var = nc.createVariable('bxs', datatype=np.float32,dimensions=('x','y','z'))
bx_var[:] = bxs
bx_var.units= 'Bifrost'

by_var = nc.createVariable('bys', datatype=np.float32,dimensions=('x','y','z'))
by_var[:] = bys
by_var.units= 'Bifrost'

bz_var = nc.createVariable('bzs', datatype=np.float32,dimensions=('x','y','z'))
bz_var[:] = bzs
bz_var.units= 'Bifrost'

bmag_var = nc.createVariable('bmag', datatype=np.float32,dimensions=('x','y','z'))
bmag_var[:] = bmag
bmag_var.units= 'Bifrost'

x_var = nc.createVariable('x', np.float32, ('x',))
x_var[:] = x
x_var.units = 'Mm'
x_var.axis = 'X' # Optional

y_var = nc.createVariable('y', np.float32, ('y',))
y_var[:] = y
y_var.units = 'Mm'
y_var.axis = 'Y' # Optional

z_var = nc.createVariable('z', np.float32, ('z',))
z_var[:] = z
z_var.units = 'Mm'
z_var.axis = 'Z' # Optional

nc.close()